In [1]:
import modelskill as ms
import mikeio1d as m1d
import numpy as np

## Current workflow

In [2]:
# - Loading model_results
path_to_res1d = '../tests/testdata/network.res1d'
nt = m1d.open(path_to_res1d)

# Loading observations
# - We fabricate an observation set to try the current workflow
np.random.seed(42)

observations = nt.read()
observations = observations + np.random.normal(0, 10, observations.shape)
# Arbitrarily selecting 2 columns of WaterLevel that will be used as observations
relevant_columns = [col for col in observations.columns if "WaterLevel" in col][8:10]
observations = observations.loc[:, relevant_columns].rename(columns=lambda x: "sensor_" + x.split(":")[1])
observations = observations.resample("1min").mean()

In [3]:
obs_9 = ms.PointObservation(observations, item="sensor_9")
obs_10 = ms.PointObservation(observations, item="sensor_10")

res1d_item = "WaterLevel:7"
nt = m1d.open(path_to_res1d)
df = nt.to_dataframe()
modres = ms.PointModelResult(df, item=res1d_item)

comparer_1 = ms.match(obs_9, modres)
comparer_2 = ms.match(obs_10, modres)

ccol0 = ms.ComparerCollection([comparer_1, comparer_2])
ccol0.skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068
sensor_10,110,0.535862,9.336934,9.321544,7.807391,0.017057,0.048156,-0.003818


- We can simplify further the workflow and not initialize `PointObservation` and `PointModelResult`.

The challenge is the 1d matching (?)

- [Design 1](https://github.com/DHI/modelskill/pull/536) assumes the user knows the sensor location in `mikeio1d` _coordinates_ (reach, chainage, node, catchment).
- [Design 2](https://github.com/DHI/modelskill/pull/536) suggests a more general approach where we would need to define a __shared__ coordinate system that represents a network architecture. We would need a plugin for each potential model data source (`mikeio1d`, `EPANET`, surrogates)

### Design 1

In [4]:
res = ms.NetworkModelResult(path_to_res1d, "Water Level", node=7)
obs_1 = ms.PointObservation(observations, item="sensor_9")

comparer_1 = ms.match(obs_1, res)
comparer_1.skill()

C:\Users\japr\Repos\modelskill\src\modelskill\quantity.py:190: UserWarning: unit='meter' was automatically set for type_name='Water Level'
  warnings.warn(f"{unit=} was automatically set for {type_name=}")


,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068


- Multiple observations with one result

In [5]:
quantity = "Water Level"
mod_item = ms.NetworkModelResult(path_to_res1d, quantity, node=7)
sens = [
    ms.PointObservation(observations, item="sensor_9"),
    ms.PointObservation(observations, item="sensor_10"),
]

cmp = ms.match(sens, mod_item)
cmp.skill()

C:\Users\japr\Repos\modelskill\src\modelskill\quantity.py:190: UserWarning: unit='meter' was automatically set for type_name='Water Level'
  warnings.warn(f"{unit=} was automatically set for {type_name=}")


,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068
sensor_10,110,0.535862,9.336934,9.321544,7.807391,0.017057,0.048156,-0.003818


In [ ]:
quantity = "Water Level"
mod_item = ms.NetworkModelResult(path_to_res1d, quantity, node=7)
sens = [
    ms.PointObservation(observations, item="sensor_9"),
    ms.PointObservation(observations, item="sensor_10"),
]

cmp = ms.match(sens, mod_item)
cmp.skill()

- Multiple models with one observation

In [6]:
quantity = ms.Quantity("Water Level", unit="m3_per_sec")
models = [
    ms.NetworkModelResult(path_to_res1d, quantity, node=7, name="model 1"),
    ms.NetworkModelResult(path_to_res1d, quantity, reach="100l1", gridpoint="start", name="model 2"),
    ms.NetworkModelResult(path_to_res1d, quantity, reach="54l1", gridpoint=0, name="model 3"),
    ms.NetworkModelResult(path_to_res1d, reach="54l1", gridpoint=0, name="model 4")
]

obs1 = ms.PointObservation(observations, quantity=quantity, item="sensor_9")

comparer_1 = ms.match(obs1, models)
comparer_1.skill()

,,n,bias,rmse,urmse,mae,cc,si,r2
model,observation,,,,,,,,
model 1,sensor_9,110,0.743778,9.167965,9.137744,7.701389,0.181837,0.047257,0.008068
model 2,sensor_9,110,2.248587,9.408963,9.136325,7.882746,0.205237,0.047249,-0.044767
model 3,sensor_9,110,0.808813,9.240242,9.204776,7.772271,0.212348,0.047603,-0.007633
model 4,sensor_9,110,0.808813,9.240242,9.204776,7.772271,0.212348,0.047603,-0.007633
